In [3]:
import pandas as pd
import numpy as np
from pppw import PPPW_Generator, init_params
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

data_source = 'ktv'

In [4]:
print('load '+ data_source + ' ...')
ktv = pd.read_csv('C:/Users/iZabett/Downloads/part-00000-8e7df0ed-7e74-476d-8645-f244ca0db1e6-c000.csv')
ktv = ktv[:10000] # notebook will process on first 10000 of entire 3+ million records
# mask preview
ktv_mask = ktv[:5].copy(); ktv_mask['patientId'] = -99999; ktv_mask.head()

load ktv ...


,year,month,patientId,provfs,Fistpatients11,AVF,AVF_f,age18to24,age25to59,age60to74,agege75,bmilt185,bmi185to249,bmigt249,inNH_prevYr,ESRD_vintagele1,ESRD_vintage1to5,ESRD_vintage5to9,ESRD_vintagegt9,pre_NEPHCARE,prim_diab,noprim_diab,CHF,CAD,CVA,PVASC,PULMON,DRUG,inc_noambtrn,Anemia,NonVAInf,medcov_6m
0,2019,7,-99999,502571,1,1,0.760085,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1
1,2019,5,-99999,142523,1,1,0.695918,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1
2,2019,6,-99999,552688,1,1,0.701548,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0
3,2019,1,-99999,212574,1,1,0.629717,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1
4,2019,4,-99999,552622,1,1,0.661743,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1


In [5]:
print('Delete unnecessary columns:')
del ktv['AVF']; del ktv['AVF_f']; del ktv['bmilt185']; del ktv['bmi185to249'];
del ktv['bmigt249']; del ktv['inNH_prevYr']; del ktv['ESRD_vintagele1'];
del ktv['ESRD_vintage1to5']; del ktv['ESRD_vintage5to9']; del ktv['ESRD_vintagegt9'];
del ktv['pre_NEPHCARE']; del ktv['prim_diab']; del ktv['noprim_diab'];
del ktv['CHF']; del ktv['CAD']; del ktv['CVA'];
del ktv['PVASC']; del ktv['PULMON']; del ktv['DRUG'];
del ktv['inc_noambtrn']; del ktv['Anemia']; del ktv['NonVAInf'];
del ktv['medcov_6m']
# mask preview
ktv_mask = ktv[:5].copy(); ktv_mask['patientId'] = -99999; ktv_mask.head()

Delete unnecessary columns:


,year,month,patientId,provfs,Fistpatients11,age18to24,age25to59,age60to74,agege75
0,2019,7,-99999,502571,1,0,0,0,1
1,2019,5,-99999,142523,1,0,1,0,0
2,2019,6,-99999,552688,1,0,0,1,0
3,2019,1,-99999,212574,1,0,0,1,0
4,2019,4,-99999,552622,1,0,0,1,0


In [6]:
# loading other dataset
all_bd = pd.read_csv('C:/Users/iZabett/Downloads/PPPW/All_Pid_BD.csv')
# unos = pd.read_csv('C:/Users/iZabett/Downloads/PPPW/unos_new.csv')
unos_org = pd.read_csv('C:/Users/iZabett/Downloads/PPPW/unos_waitg_lst_202009221849.csv', delimiter='|')
# unos_new = pd.read_csv('C:/Users/iZabett/Downloads/PPPW/unos_new.csv')


In [7]:
# preview patient-DOB data
# mask preview
bd_mask = all_bd[:5].copy(); bd_mask['patient_id'] = -99999; bd_mask.head()

,patient_id,Date_of_Birth
0,-99999,6/27/1962
1,-99999,4/5/1941
2,-99999,5/28/1962
3,-99999,7/6/1945
4,-99999,7/26/1957


In [8]:
# preview UNOS data
# mask preview
unos_mask = unos_org[:5].copy(); unos_mask['ptnt_id'] = -99999; unos_mask['unos_waitg_lst_id'] = -88888; unos_mask.head()

,unos_waitg_lst_id,ptnt_id,lstg_dt,rmvd_dt,unos_waitg_lst_orgn_cd,birth_dt,death_dt
0,-88888,-99999,2018-08-08,NaN,KI,1949-07-19,NaN
1,-88888,-99999,2018-02-02,2019-01-14,KI,1982-04-09,NaN
2,-88888,-99999,2019-10-25,NaN,KI,1955-05-07,NaN
3,-88888,-99999,2019-04-16,NaN,KI,1955-02-15,NaN
4,-88888,-99999,2016-11-28,2019-04-19,KI,1951-03-10,NaN


In [9]:
# adding auxiliary columns and DOB from all_pid_bd.csv to ktv.csv

# ktv_all = pd.read_csv('C:/Users/iZabett/Downloads/PPPW/KtV_1.csv')
ktv_all = ktv
ktv_all['unos_waitg_lst_id'] = None
ktv_all['lstg_dt'] = None
ktv_all['rmvd_dt'] = None
ktv_all['unos_waitg_lst_orgn_cd'] = None
ktv_all['birth_date'] = None
ktv_all['death_dt'] = None
ktv_all['W_End'] = None
cols = ['patientId', 'provfs', 'year', 'month', 'unos_waitg_lst_id', 'lstg_dt', 'rmvd_dt', 'unos_waitg_lst_orgn_cd'
    , 'Fistpatients11', 'age18to24', 'age25to59', 'age60to74', 'agege75', 'birth_date', 'death_dt', 'W_End']
ktv_all = ktv_all[cols]
# remove pid records of ktv which are not in unos
# ktv_all = ktv_all[ktv_all['patientId'].isin(list(unos['ptnt_id']))]
# ktv_all = ktv_all[:32500]

# mask preview
ktv_mask = ktv_all[:5].copy(); ktv_mask['patientId'] = -99999; ktv_mask.head()

,patientId,provfs,year,month,unos_waitg_lst_id,lstg_dt,rmvd_dt,unos_waitg_lst_orgn_cd,Fistpatients11,age18to24,age25to59,age60to74,agege75,birth_date,death_dt,W_End
0,-99999,502571,2019,7,None,None,None,None,1,0,0,0,1,None,None,None
1,-99999,142523,2019,5,None,None,None,None,1,0,1,0,0,None,None,None
2,-99999,552688,2019,6,None,None,None,None,1,0,0,1,0,None,None,None
3,-99999,212574,2019,1,None,None,None,None,1,0,0,1,0,None,None,None
4,-99999,552622,2019,4,None,None,None,None,1,0,0,1,0,None,None,None


In [10]:
# preprocessing ...
pppw_gen = PPPW_Generator(**init_params(data=ktv_all, unos=unos_org, bdate=all_bd))
# mask preview
ktv_mask = pppw_gen.data[:5].copy(); ktv_mask['patientId'] = -99999; ktv_mask.head()

  0%|▊                                                                                                                                                                                                                     | 57/16072 [00:00<00:28, 564.36it/s]

unos_prep ...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16072/16072 [00:39<00:00, 409.53it/s]
20it [00:00, 198.01it/s]

unos_merge ...



2500it [00:13, 181.99it/s]
18it [00:00, 168.22it/s]

2500it [00:12, 206.89it/s]
46it [00:00, 224.83it/s]

2500it [00:12, 199.97it/s]
21it [00:00, 209.99it/s]

2500it [00:12, 207.73it/s]
0it [00:00, ?it/s]


,Unnamed: 0,patientId,provfs,year,month,unos_waitg_lst_id,lstg_dt,rmvd_dt,unos_waitg_lst_orgn_cd,Fistpatients11,age18to24,age25to59,age60to74,agege75,birth_date,death_dt,W_End
0,0,-99999,502571,2019,7,NaN,NaN,NaN,NaN,1,0,0,0,1,1/23/1944,NaN,NaN
1,1,-99999,142523,2019,5,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN,NaN,NaN
2,2,-99999,552688,2019,6,NaN,NaN,NaN,NaN,1,0,0,1,0,10/7/1955,NaN,NaN
3,3,-99999,212574,2019,1,NaN,NaN,NaN,NaN,1,0,0,1,0,12/23/1954,NaN,NaN
4,4,-99999,552622,2019,4,NaN,NaN,NaN,NaN,1,0,0,1,0,10/1/1955,NaN,NaN


In [11]:
# some filtering on patient watilist data 

# pw_all =  pd.read_csv('C:/Users/iZabett/Downloads/ktv_prep_all.csv')
pw_all = pppw_gen.data

# Filter out Fistpatients11 Flag==0
pw_all = pw_all[pw_all['Fistpatients11'] == 1]
# Filter out agege75 Flag==1 (meaning age>75)
pw_all = pw_all[pw_all['agege75'] == 0]

# Reset index
pw_all = pw_all.reset_index()
cols = ['patientId', 'provfs', 'year', 'month', 'birth_date',  'W_End']
pw_all = pw_all[cols]

In [12]:
# main processing on data
pppw_gen.processings(pw_all)

108it [00:00, 1074.34it/s]

age calculations ...


2500it [00:01, 1666.05it/s]
2500it [00:01, 1506.94it/s]
2500it [00:01, 1642.58it/s]
251it [00:00, 1662.23it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

Age Calculation is done with 2090 exceptions
waitlist calculations ...


2500it [00:04, 511.88it/s]
2500it [00:04, 545.61it/s]
2500it [00:04, 558.02it/s]
251it [00:00, 484.56it/s]
0it [00:00, ?it/s]


In [13]:
# mask preview of calculation Waitlist_Flag
ktv_mask = pppw_gen.data[:5].copy(); ktv_mask['patientId'] = -99999; ktv_mask.head()

,patientId,provfs,year,month,W_End,Age,Waitlist_Flag,Aux_Wl_Flag
0,-99999,552688,2019,6,NaN,63.0,0,NaN
1,-99999,212574,2019,1,NaN,64.0,0,NaN
2,-99999,552622,2019,4,NaN,63.0,0,NaN
3,-99999,552667,2019,11,NaN,48.0,0,NaN
4,-99999,212638,2019,4,NaN,49.0,0,NaN


In [ ]:
# write final data
pppw_gen.data.to_csv(pppw_gen.data_path + data_source + '_final.csv')